In [1]:
pip install Pyomo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.6 MB/s eta 0:00:00


In [2]:
! apt install libglpk-dev python3.8-dev libgmp3-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libbtf1 libcamd2 libccolamd2 libcholmod3 libcolamd2 libcxsparse3 libglpk40 libgmp-dev
  libgmpxx4ldbl libgraphblas-dev libgraphblas6 libklu1 libldl2 libmetis5 libmongoose2 libpython3.8
  libpython3.8-dev libpython3.8-minimal libpython3.8-stdlib librbio2 libsliplu1 libspqr2
  libsuitesparse-dev libsuitesparseconfig5 libumfpack5 mailcap mime-support python3.8
  python3.8-minimal
Suggested packages:
  libiodbc2-dev gmp-doc libgmp10-doc libmpfr-dev python3.8-venv binfmt-support
The following NEW packages will be installed:
  libamd2 libbtf1 libcamd2 libccolamd2 libcholmod3 libcolamd2 libcxsparse3 libglpk-dev libglpk40
  libgmp-dev libgmp3-dev libgmpxx4ldbl libgraphblas-dev libgraphblas6 libklu1 libldl2 libmetis5
  libmongoose2 libpython3.8 libpython3.8-dev libpython3.8-minimal libpython3.8-stdlib librbio2
  libsliplu1 libspqr2 libsu

In [3]:
!pip install pyomo
!apt install glpk-utils
!pip install glpk

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  glpk-utils
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 214 kB of archives.
After this operation, 1,042 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 glpk-utils amd64 5.0-1 [214 kB]
Fetched 214 kB in 0s (1,606 kB/s)
Selecting previously unselected package glpk-utils.
(Reading database ... 122990 files and directories currently installed.)
Preparing to unpack .../glpk-utils_5.0-1_amd64.deb ...
Unpacking glpk-utils (5.0-1) ...
Setting up glpk-utils (5.0-1) ...
Processing triggers for man-db (2.10.2-1) ...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.2/161.2 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for glpk: filename=glpk-0.

In [4]:
import glpk

In [5]:
import numpy as np
import scipy.optimize
import pandas as pd
import pyomo.environ as pyo


In [6]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier, GradientBoostingClassifier, VotingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, SGDRegressor
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, classification_report, confusion_matrix, mean_squared_error, r2_score, hamming_loss

In [7]:
sales = pd.read_csv("Sales TEC_Valid.csv")

In [8]:
# Filtrar para solo quedarnos con las ventas físicas de comida (solo las columnas que son relevantes para el problema de abastecimiento)
food_types = ("Botanas", "Refrescos", "Licores", "Perecederos", "Galletas", "Bebidas Calientes", "Sopas", "Lacteos", "Alimentos Charter")
food_sales = sales[sales["ProductType"].isin(food_types)]
food_sales.drop(["ProductType", "TotalSales"], axis = 1, inplace=True)

# Pivotear la tabla de tal manera que cada fila corresponda a un vuelo, y posteriormente ordenar de productos más populares a menos populares
food_sales = food_sales.pivot(index="Flight_ID", columns="ProductName", values="Quantity").fillna(0)
food_sales = food_sales[food_sales.sum().sort_values(ascending=False).index]

food_sales

<ipython-input-8-8a791f6fdd1a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  food_sales.drop(["ProductType", "TotalSales"], axis = 1, inplace=True)


ProductName,Coca Cola Regular,Agua Natural 600 Ml,Sabritas Originales,Ruffles Queso,Doritos Nacho,Panini Clasico,Fritos Limon Y Sal,Sabritas Flamin Hot,Sprite,Cheetos,...,Maxi Combo,Heineken 0,Cuerno Individual Charter,Quaker Natural Balance,Quaker Avena Moras,Nissin Limon Y Habanero,Combo Stl,Club Sandwich,Kacang Flaming Hot,Gomita Enchilada La Cueva
Flight_ID,,,,,,,,,,,,,,,,,,,,,
00004a718edba9d9ef878d08f02ae057,6.0,4.0,4.0,4.0,1.0,3.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0000cd79c0c3a9c309df6064dcacaeea,10.0,2.0,2.0,0.0,7.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000163f0df9cbfc35c4c06645ec512f6,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00017be73003a570dd426b155762769c,11.0,8.0,12.0,6.0,4.0,6.0,4.0,5.0,8.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0001a43836c338f8d8650aefb11672c9,11.0,3.0,3.0,4.0,7.0,3.0,3.0,4.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fffe47f84ecc55da94b2907a7317dd12,14.0,15.0,0.0,8.0,11.0,0.0,7.0,0.0,5.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
fffe4a9c19ae2320e76f02939fdff957,7.0,4.0,4.0,2.0,3.0,1.0,4.0,0.0,4.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ffff138536f249f69340e0a8336f94a4,18.0,5.0,2.0,7.0,8.0,0.0,3.0,3.0,1.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
flights = pd.read_csv("Filghts TEC_Valid.csv")

In [10]:
# Quitar los registros con valores NaN (aquellos para predecir en el 2024)
flights = flights.dropna()
flights

,Flight_ID,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Passengers,Bookings
0,ab954014077430bd842cfa305a55c0f8,XA-VBY,AT,AZ,Ciudad Fronteriza,Ciudad Principal,2023-10-19 11:40:00,2023-10-19 14:25:00,240,229.0,157.0
1,efd86c996035dacdca7a0ccb2560dda1,XA-VIX,BM,AV,MX Amigos y Familia,Ciudad Fronteriza,2023-07-03 00:55:00,2023-07-03 04:55:00,186,197.0,109.0
3,dd0fad3248951d2f71d63e6279aeaa4b,XA-VBW,AW,AS,MX Amigos y Familia,Ciudad Principal,2023-06-26 15:15:00,2023-06-26 15:55:00,220,200.0,142.0
4,d0987ee648eea254063bfe2b39571b67,XA-VAP,BA,AB,Playa,Ciudad Principal,2023-02-10 08:40:00,2023-02-10 09:50:00,186,162.0,90.0
5,3b5df8805161ea827d2f2e4298c38e06,XA-VBY,AJ,AR,Playa,MX Amigos y Familia,2023-09-07 17:10:00,2023-09-07 18:05:00,240,183.0,125.0
...,...,...,...,...,...,...,...,...,...,...,...
245718,d9ef769565064b1ff1ad90d6d18f69cb,XA-VYF,AO,BT,Playa,Ciudad Principal,2023-12-26 06:20:00,2023-12-26 10:10:00,180,166.0,89.0
245745,598035669a4f0fa2ce1207977932df9e,XA-VAI,AW,BT,Playa,Ciudad Principal,2023-12-28 10:25:00,2023-12-28 13:50:00,180,142.0,54.0
245747,90d2cab80b7700b5d854ae393678144b,XA-VYA,BA,BT,Playa,Ciudad Principal,2023-12-23 11:45:00,2023-12-23 15:00:00,180,156.0,61.0
245748,cc7c1c5e6fd132fd0bdab3a35aac33c0,XA-VBK,BM,BT,Playa,Ciudad Fronteriza,2023-12-29 07:30:00,2023-12-29 14:50:00,240,189.0,86.0


In [11]:
# Combinar ambos dataframes de acuerdo con su Flight_ID, y rellenar los valores faltantes con 0 (no hubo venta de ese producto en ese vuelo)
df = flights.set_index("Flight_ID").join(food_sales)
df = df.fillna(0)

# Convertir las fechas a datetime para manejar con mayor facilidad
df["STD"] = pd.to_datetime(df["STD"])
df["STA"] = pd.to_datetime(df["STA"])

# Crear nuevas columnas numéricas para mes, día, y hora tanto para origen como destino. Crear una nueva columna que combine ciudad de origen y destino
new_cols = pd.DataFrame({
    'Departure Month' : df['STD'].dt.month,
    'Departure Day' : df['STD'].dt.day,
    'Departure Time' : df['STD'].dt.hour + df['STD'].dt.minute / 60,
    'Arrival Month' : df['STA'].dt.month,
    'Arrival Day' : df['STA'].dt.day,
    'Arrival Time' : df['STA'].dt.hour + df['STA'].dt.minute / 60,
    'DS-AS': df['DepartureStation'] + "-" + df["ArrivalStation"]
}, index=df.index)
df = pd.concat([new_cols, df], axis = 1)

# Quitamos los vuelos cuya ruta se voló menos de 50 veces para facilitar subsampling estratificado. Estos constituyen menos del 1% de los vuelos totales
to_remove = df['DS-AS'].value_counts()[df['DS-AS'].value_counts() < 50].index
df = df[~df['DS-AS'].isin(to_remove)]

# Baileys y Jw Red Label tienen columnas duplicadas debido a un error de ortografía. Combinamos estas columnas
df["Baileys"] = df["Baileys"] + df["Baileys "]
df["Jw Red Label"] = df["Jw Red Label"] + df["Jw Red Label "]

# Quitamos columnas no utiles o no usadas para mejorar el desempeño de los modelos, además de las columnas duplicadas
df.drop(["Aeronave", "Bookings", "STD", "STA", "Baileys ", "Jw Red Label "], axis = 1, inplace = True)

df

<ipython-input-11-3067fd73083f>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Baileys"] = df["Baileys"] + df["Baileys "]
<ipython-input-11-3067fd73083f>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Jw Red Label"] = df["Jw Red Label"] + df["Jw Red Label "]
<ipython-input-11-3067fd73083f>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ret

,Departure Month,Departure Day,Departure Time,Arrival Month,Arrival Day,Arrival Time,DS-AS,DepartureStation,ArrivalStation,Destination_Type,...,Maxi Combo,Heineken 0,Cuerno Individual Charter,Quaker Natural Balance,Quaker Avena Moras,Nissin Limon Y Habanero,Combo Stl,Club Sandwich,Kacang Flaming Hot,Gomita Enchilada La Cueva
Flight_ID,,,,,,,,,,,,,,,,,,,,,
00004a718edba9d9ef878d08f02ae057,12,18,16.333333,12,18,18.583333,AL-AT,AL,AT,Ciudad Principal,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0000cd79c0c3a9c309df6064dcacaeea,10,7,18.916667,10,7,20.500000,AK-AD,AK,AD,MX Amigos y Familia,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00010d4c88e2cb6089937fd4e4f9783b,7,26,13.583333,7,26,14.500000,AO-BD,AO,BD,Playa,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000163f0df9cbfc35c4c06645ec512f6,10,29,6.083333,10,29,7.500000,AW-BF,AW,BF,MX Amigos y Familia,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00017be73003a570dd426b155762769c,10,17,9.333333,10,17,13.083333,BM-BD,BM,BD,Playa,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fffe47f84ecc55da94b2907a7317dd12,6,7,15.000000,6,7,17.250000,BH-BM,BH,BM,Ciudad Fronteriza,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
fffe4a9c19ae2320e76f02939fdff957,7,22,16.250000,7,22,18.000000,AT-AW,AT,AW,Ciudad Principal,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ffff138536f249f69340e0a8336f94a4,6,11,14.000000,6,11,17.333333,AW-AK,AW,AK,Playa,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# Creamos 6 bins (0, 1-4, 5-8, 9-12, 13-20, 20-40, 41-100) para clasificar los valores de los productos vendidos.
bins = [-np.inf, 0, 4, 8, 12, 20, 40, 100]
labels = ["0", "4", "8", "12", "20", "40", "100"]
for col in df.columns[-98:]:
    df[col] = pd.cut(df[col], bins = bins, labels = labels, right=True, include_lowest=True)
df

,Departure Month,Departure Day,Departure Time,Arrival Month,Arrival Day,Arrival Time,DS-AS,DepartureStation,ArrivalStation,Destination_Type,...,Maxi Combo,Heineken 0,Cuerno Individual Charter,Quaker Natural Balance,Quaker Avena Moras,Nissin Limon Y Habanero,Combo Stl,Club Sandwich,Kacang Flaming Hot,Gomita Enchilada La Cueva
Flight_ID,,,,,,,,,,,,,,,,,,,,,
00004a718edba9d9ef878d08f02ae057,12,18,16.333333,12,18,18.583333,AL-AT,AL,AT,Ciudad Principal,...,0,0,0,0,0,0,0,0,0,0
0000cd79c0c3a9c309df6064dcacaeea,10,7,18.916667,10,7,20.500000,AK-AD,AK,AD,MX Amigos y Familia,...,0,0,0,0,0,0,0,0,0,0
00010d4c88e2cb6089937fd4e4f9783b,7,26,13.583333,7,26,14.500000,AO-BD,AO,BD,Playa,...,0,0,0,0,0,0,0,0,0,0
000163f0df9cbfc35c4c06645ec512f6,10,29,6.083333,10,29,7.500000,AW-BF,AW,BF,MX Amigos y Familia,...,0,0,0,0,0,0,0,0,0,0
00017be73003a570dd426b155762769c,10,17,9.333333,10,17,13.083333,BM-BD,BM,BD,Playa,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fffe47f84ecc55da94b2907a7317dd12,6,7,15.000000,6,7,17.250000,BH-BM,BH,BM,Ciudad Fronteriza,...,0,0,0,0,0,0,0,0,0,0
fffe4a9c19ae2320e76f02939fdff957,7,22,16.250000,7,22,18.000000,AT-AW,AT,AW,Ciudad Principal,...,0,0,0,0,0,0,0,0,0,0
ffff138536f249f69340e0a8336f94a4,6,11,14.000000,6,11,17.333333,AW-AK,AW,AK,Playa,...,0,0,0,0,0,0,0,0,0,0


In [13]:
# Aplicar subsampling estratificado con respecto a la ruta (para garantizar que se aprende sobre todas las rutas)
sampled_df = df.groupby('DS-AS').apply(lambda x: x.sample(frac=0.1)).reset_index(drop=True)

In [14]:
X_p = sampled_df.iloc[:,:12]
y_p = sampled_df.iloc[:,12]
X_p

,Departure Month,Departure Day,Departure Time,Arrival Month,Arrival Day,Arrival Time,DS-AS,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,Capacity
0,3,9,20.416667,3,9,23.916667,AB-AK,AB,AK,Playa,Playa,186
1,2,26,8.833333,2,26,12.333333,AB-AK,AB,AK,Playa,Playa,186
2,3,19,8.833333,3,19,12.333333,AB-AK,AB,AK,Playa,Playa,186
3,2,19,8.833333,2,19,12.333333,AB-AK,AB,AK,Playa,Playa,186
4,6,5,14.583333,6,5,18.000000,AB-AK,AB,AK,Playa,Playa,186
...,...,...,...,...,...,...,...,...,...,...,...,...
12079,9,3,10.166667,9,3,11.500000,BS-BA,BS,BA,Ciudad Principal,Playa,186
12080,8,27,10.166667,8,27,11.500000,BS-BA,BS,BA,Ciudad Principal,Playa,186
12081,9,17,10.166667,9,17,11.500000,BS-BA,BS,BA,Ciudad Principal,Playa,186
12082,10,8,10.250000,10,8,11.500000,BS-BA,BS,BA,Ciudad Principal,Playa,186


In [15]:
# Definimos las variables categóricas para aplicarles distintas herramientas de preprocesamiento
categorical_features_p = ['DepartureStation', 'ArrivalStation','Destination_Type','Origin_Type','DS-AS']
numerical_features_p = ['Departure Month', 'Departure Day', 'Departure Time', 'Arrival Month', 'Arrival Day', 'Arrival Time', 'Capacity']

# Aplicamos One Hot Encoding a las variables categóricas para poder utilizarlas en los modelos de clasificación
preprocessor_p = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numerical_features_p),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features_p)
    ])

# Hacemos un split 80-20 para el entrenamiento y la prueba
X_train_p, X_test_p, y_train_p, y_test_p = train_test_split(X_p, y_p, test_size=0.2)

In [16]:
# Creamos una pipeline para que los datos sean procesados y luego ajustados a un Random Forest Regressor
model_rf_p = Pipeline(steps=[
    ('preprocessor', preprocessor_p),
    ('regressor', RandomForestRegressor())
])
model_rf_p.fit(X_train_p, y_train_p)

# Hacemos predicciones con los valores de prueba
y_pred_rf_p = model_rf_p.predict(X_test_p)

In [17]:
# Revisamos las métricas de Mean Squared Error (menor es mejor) y r2
print(mean_squared_error(y_test_p.to_numpy(), y_pred_rf_p))
print(r2_score(y_test_p.to_numpy(), y_pred_rf_p))

717.8857690525444
0.5202379387530772


In [18]:
# Creamos una pipeline para que los datos sean procesados y luego ajustados a un K Neighbors Regressor
model_kn_p = Pipeline(steps=[
    ('preprocessor', preprocessor_p),
    ('regressor', KNeighborsRegressor())
])
model_kn_p.fit(X_train_p, y_train_p)

# Hacemos predicciones con los valores de prueba
y_pred_kn_p = model_kn_p.predict(X_test_p)

In [19]:
# Revisamos las métricas de Mean Squared Error (menor es mejor) y r2
print(mean_squared_error(y_test_p.to_numpy(), y_pred_kn_p))
print(r2_score(y_test_p.to_numpy(), y_pred_kn_p))

814.9144062887877
0.4553938354051631


In [20]:
# Creamos una pipeline para que los datos sean procesados y luego ajustados a un Gradient Boosting Regressor
model_gb_p = Pipeline(steps=[
    ('preprocessor', preprocessor_p),
    ('regressor', GradientBoostingRegressor())
])
model_gb_p.fit(X_train_p, y_train_p)

# Hacemos predicciones con los valores de prueba
y_pred_gb_p = model_gb_p.predict(X_test_p)

In [21]:
# Revisamos las métricas de Mean Squared Error (menor es mejor) y r2
print(mean_squared_error(y_test_p.to_numpy(), y_pred_gb_p))
print(r2_score(y_test_p.to_numpy(), y_pred_gb_p))

838.6825252966454
0.4395096345214744


In [22]:
# Creamos una pipeline para que los datos sean procesados y luego ajustados a un Support Vector Regressor
model_sv_p = Pipeline(steps=[
    ('preprocessor', preprocessor_p),
    ('regressor', SVR())
])
model_sv_p.fit(X_train_p, y_train_p)

# Hacemos predicciones con los valores de prueba
y_pred_sv_p = model_sv_p.predict(X_test_p)

In [23]:
# Revisamos las métricas de Mean Squared Error (menor es mejor) y r2
print(mean_squared_error(y_test_p.to_numpy(), y_pred_sv_p))
print(r2_score(y_test_p.to_numpy(), y_pred_sv_p))

937.1248766492627
0.3737207479953387


In [24]:
# Creamos una pipeline para que los datos sean procesados y luego ajustados a un Linear Regressor
model_li_p = Pipeline(steps=[
    ('preprocessor', preprocessor_p),
    ('regressor', LinearRegression())
])
model_li_p.fit(X_train_p, y_train_p)

# Hacemos predicciones con los valores de prueba
y_pred_li_p = model_li_p.predict(X_test_p)

In [25]:
# Revisamos las métricas de Mean Squared Error (menor es mejor) y r2
print(mean_squared_error(y_test_p.to_numpy(), y_pred_li_p))
print(r2_score(y_test_p.to_numpy(), y_pred_li_p))

824.2379646013089
0.44916291434919264


In [26]:
# Creamos una pipeline para que los datos sean procesados y luego ajustados a un Ridge Regressor
model_ri_p = Pipeline(steps=[
    ('preprocessor', preprocessor_p),
    ('regressor', Ridge())
])
model_ri_p.fit(X_train_p, y_train_p)

# Hacemos predicciones con los valores de prueba
y_pred_ri_p = model_ri_p.predict(X_test_p)

In [27]:
# Revisamos las métricas de Mean Squared Error (menor es mejor) y r2
print(mean_squared_error(y_test_p.to_numpy(), y_pred_ri_p))
print(r2_score(y_test_p.to_numpy(), y_pred_ri_p))

822.3235004024747
0.45044234811123574


In [28]:
# Creamos una pipeline para que los datos sean procesados y luego ajustados a un Stochastic Gradient Descent Regressor
model_sgd_p = Pipeline(steps=[
    ('preprocessor', preprocessor_p),
    ('regressor', SGDRegressor())
])
model_sgd_p.fit(X_train_p, y_train_p)

# Hacemos predicciones con los valores de prueba
y_pred_sgd_p = model_sgd_p.predict(X_test_p)

In [29]:
# Revisamos las métricas de Mean Squared Error (menor es mejor) y r2
print(mean_squared_error(y_test_p.to_numpy(), y_pred_sgd_p))
print(r2_score(y_test_p.to_numpy(), y_pred_sgd_p))

822.277425939249
0.4504731396109435


In [30]:
# Creamos una pipeline para que los datos sean procesados y luego ajustados a un MLP Regressor
model_mlp_p = Pipeline(steps=[
    ('preprocessor', preprocessor_p),
    ('regressor', MLPRegressor())
])
model_mlp_p.fit(X_train_p, y_train_p)

# Hacemos predicciones con los valores de prueba
y_pred_mlp_p = model_mlp_p.predict(X_test_p)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [31]:
# Revisamos las métricas de Mean Squared Error (menor es mejor) y r2
print(mean_squared_error(y_test_p.to_numpy(), y_pred_mlp_p))
print(r2_score(y_test_p.to_numpy(), y_pred_mlp_p))

741.8083007651867
0.5042505440734899


In [32]:
# Dividir entre variables predictoras (X) y variables a predecir (y)
X_f = sampled_df.iloc[:, :13]
y_f = sampled_df.iloc[:, -98:]
X_f

,Departure Month,Departure Day,Departure Time,Arrival Month,Arrival Day,Arrival Time,DS-AS,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,Capacity,Passengers
0,3,9,20.416667,3,9,23.916667,AB-AK,AB,AK,Playa,Playa,186,137.0
1,2,26,8.833333,2,26,12.333333,AB-AK,AB,AK,Playa,Playa,186,160.0
2,3,19,8.833333,3,19,12.333333,AB-AK,AB,AK,Playa,Playa,186,171.0
3,2,19,8.833333,2,19,12.333333,AB-AK,AB,AK,Playa,Playa,186,160.0
4,6,5,14.583333,6,5,18.000000,AB-AK,AB,AK,Playa,Playa,186,131.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12079,9,3,10.166667,9,3,11.500000,BS-BA,BS,BA,Ciudad Principal,Playa,186,113.0
12080,8,27,10.166667,8,27,11.500000,BS-BA,BS,BA,Ciudad Principal,Playa,186,79.0
12081,9,17,10.166667,9,17,11.500000,BS-BA,BS,BA,Ciudad Principal,Playa,186,179.0
12082,10,8,10.250000,10,8,11.500000,BS-BA,BS,BA,Ciudad Principal,Playa,186,125.0


In [33]:
# Definimos las variables categóricas para aplicarles distintas herramientas de preprocesamiento
categorical_features_f = ['DepartureStation', 'ArrivalStation','Destination_Type','Origin_Type','DS-AS']
numerical_features_f = ['Departure Month', 'Departure Day', 'Departure Time', 'Arrival Month', 'Arrival Day', 'Arrival Time', 'Capacity', 'Passengers']

# Aplicamos One Hot Encoding a las variables categóricas para poder utilizarlas en los modelos de clasificación
preprocessor_f = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numerical_features_f),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features_f)
    ])

# Hacemos un split 80-20 para el entrenamiento y la prueba
X_train_f, X_test_f, y_train_f, y_test_f = train_test_split(X_f, y_f, test_size=0.2)

In [34]:
# Esta función encuentra el porcentaje de los guesses que son mayores o iguales al valor real. Esto es importante porque sobre-estimar el valor en este caso no es un problema tan grande como subestimar
def safe_score(y_true, y_pred):
    score = 0
    for i in range(len(y_true)):
        if int(y_pred[i]) >= int(y_true[i]):
            score += 1
    return score/len(y_true)

In [35]:
# Creamos una pipeline para que los datos sean procesados y luego ajustados a un Random Forest Classifier
model_rf_f = Pipeline(steps=[
    ('preprocessor', preprocessor_f),
    ('regressor', RandomForestClassifier())
])
model_rf_f.fit(X_train_f, y_train_f)

# Hacemos predicciones con los valores de prueba
y_pred_rf_f = model_rf_f.predict(X_test_f)

In [36]:
# Obtenemos el accuracy para cada columna (cada producto y lo promediamos)
accuracy_scores = [accuracy_score(y_test_f.to_numpy()[:, i], y_pred_rf_f[:, i]) for i in range(y_test_f.shape[1])]
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
print(average_accuracy)

# Obtenemos el safe score para cada columna (cada producto y lo promediamos)
safe_scores = [safe_score(y_test_f.to_numpy()[:, i], y_pred_rf_f[:, i]) for i in range(y_test_f.shape[1])]
average_safe = sum(safe_scores) / len(safe_scores)
print(average_safe)

# El safe score para los 10 productos más populares
print(safe_scores[0:10])

0.8396097371509631
0.8925552844224157
[0.6988001654944146, 0.70707488622259, 0.726934215970211, 0.7645841952834092, 0.775755068266446, 0.78113363673976, 0.7939594538684319, 0.7736863880844022, 0.784857261067439, 0.7910633016135705]


In [37]:
# Creamos una pipeline para que los datos sean procesados y luego ajustados a un Decision Tree Classifier
model_dt_f = Pipeline(steps=[
    ('preprocessor', preprocessor_f),
    ('regressor', DecisionTreeClassifier())
])
model_dt_f.fit(X_train_f, y_train_f)

# Hacemos predicciones con los valores de prueba
y_pred_dt_f = model_dt_f.predict(X_test_f)

In [38]:
# Obtenemos el accuracy para cada columna (cada producto y lo promediamos)
accuracy_scores = [accuracy_score(y_test_f.to_numpy()[:, i], y_pred_dt_f[:, i]) for i in range(y_test_f.shape[1])]
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
print(average_accuracy)

# Obtenemos el safe score para cada columna (cada producto y lo promediamos)
safe_scores = [safe_score(y_test_f.to_numpy()[:, i], y_pred_dt_f[:, i]) for i in range(y_test_f.shape[1])]
average_safe = sum(safe_scores) / len(safe_scores)
print(average_safe)

# El safe score para los 10 productos más populares
print(safe_scores[0:10])

0.8005412342843632
0.9014928271681035
[0.6743897393462971, 0.6996276375672321, 0.7170045510964005, 0.7654116673562267, 0.7662391394290443, 0.7778237484484899, 0.788580885395118, 0.7782374844848986, 0.790235829540753, 0.80471659081506]


In [39]:
# Creamos una pipeline para que los datos sean procesados y luego ajustados a un K Neighbors Classifier
model_kn_f = Pipeline(steps=[
    ('preprocessor', preprocessor_f),
    ('regressor', KNeighborsClassifier())
])
model_kn_f.fit(X_train_f, y_train_f)

# Hacemos predicciones con los valores de prueba
y_pred_kn_f = model_kn_f.predict(X_test_f)

In [40]:
# Obtenemos el accuracy para cada columna (cada producto y lo promediamos)
accuracy_scores = [accuracy_score(y_test_f.to_numpy()[:, i], y_pred_kn_f[:, i]) for i in range(y_test_f.shape[1])]
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
print(average_accuracy)

# Obtenemos el safe score para cada columna (cada producto y lo promediamos)
safe_scores = [safe_score(y_test_f.to_numpy()[:, i], y_pred_kn_f[:, i]) for i in range(y_test_f.shape[1])]
average_safe = sum(safe_scores) / len(safe_scores)
print(average_safe)

# El safe score para los 10 productos más populares
print(safe_scores[0:10])

0.8337118877339929
0.8905921491476193
[0.6760446834919321, 0.6847331402565163, 0.7041787339677286, 0.7372776168804303, 0.7625155151013653, 0.7736863880844022, 0.7823748448489863, 0.7525858502275549, 0.7782374844848986, 0.7803061646669425]


In [41]:
future_flights = pd.read_csv("Filghts TEC_Valid.csv")
future_flights = future_flights[future_flights["Passengers"].isna()]
future_flights

,Flight_ID,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Passengers,Bookings
2,6cfa1bbaa44f08fc7d3061f034a6a5ce,XA-VBV,AW,AS,MX Amigos y Familia,Ciudad Principal,2024-02-16 17:10:00,2024-02-16 17:55:00,220,NaN,NaN
19,bb319ee8f62b5ae38e6f6da4dbff03cb,XA-VAV,BA,AB,Playa,Ciudad Principal,2024-01-12 08:20:00,2024-01-12 09:20:00,186,NaN,NaN
22,d77570d39116f81dbb619c5225dd0572,NaN,AK,BO,MX Amigos y Familia,Playa,2024-05-18 14:10:00,2024-05-18 16:15:00,180,NaN,NaN
25,85d3b182431d323b856e50e249689981,XA-VIY,AT,AI,Playa,Ciudad Principal,2024-03-18 10:30:00,2024-03-18 13:25:00,186,NaN,NaN
26,04e83b220f5513bd6cc636784dcddec6,XA-VXC,BM,AV,MX Amigos y Familia,Ciudad Fronteriza,2024-04-05 00:50:00,2024-04-05 04:45:00,240,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
245744,a0ee9f7974186536fc197f59353a6442,XA-VAE,AO,BT,Playa,Ciudad Principal,2024-03-21 06:20:00,2024-03-21 10:10:00,186,NaN,NaN
245746,ab5bb628f7e10e2d8fbdf9e15de7bf4a,NaN,BA,BT,Playa,Ciudad Principal,2024-10-01 11:55:00,2024-10-01 15:10:00,186,NaN,NaN
245749,ef32da2731db80faa8b9f5030979a016,9H-MLV,AW,BT,Playa,Ciudad Principal,2024-04-20 10:45:00,2024-04-20 14:00:00,178,NaN,NaN
245751,3c15779202b13158f78e8a8afe377250,NaN,AW,BT,Playa,Ciudad Principal,2024-10-04 10:00:00,2024-10-04 13:15:00,240,NaN,NaN


In [42]:
# Convertir las fechas a datetime para manejar con mayor facilidad
future_flights["STD"] = pd.to_datetime(future_flights["STD"])
future_flights["STA"] = pd.to_datetime(future_flights["STA"])

# Crear nuevas columnas numéricas para mes, día, y hora tanto para origen como destino. Crear una nueva columna que combine ciudad de origen y destino
new_cols = pd.DataFrame({
    'Departure Month' : future_flights['STD'].dt.month,
    'Departure Day' : future_flights['STD'].dt.day,
    'Departure Time' : future_flights['STD'].dt.hour + future_flights['STD'].dt.minute / 60,
    'Arrival Month' : future_flights['STA'].dt.month,
    'Arrival Day' : future_flights['STA'].dt.day,
    'Arrival Time' : future_flights['STA'].dt.hour + future_flights['STA'].dt.minute / 60,
    'DS-AS': future_flights['DepartureStation'] + "-" + future_flights["ArrivalStation"]
}, index=future_flights.index)
future_flights = pd.concat([new_cols, future_flights], axis = 1)

# Quitamos columnas no utiles o no usadas para mejorar el desempeño de los modelos, además de las columnas duplicadas
future_flights.drop(["Bookings", "STD", "STA", "Passengers"], axis = 1, inplace = True)

future_flights

,Departure Month,Departure Day,Departure Time,Arrival Month,Arrival Day,Arrival Time,DS-AS,Flight_ID,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,Capacity
2,2,16,17.166667,2,16,17.916667,AW-AS,6cfa1bbaa44f08fc7d3061f034a6a5ce,XA-VBV,AW,AS,MX Amigos y Familia,Ciudad Principal,220
19,1,12,8.333333,1,12,9.333333,BA-AB,bb319ee8f62b5ae38e6f6da4dbff03cb,XA-VAV,BA,AB,Playa,Ciudad Principal,186
22,5,18,14.166667,5,18,16.250000,AK-BO,d77570d39116f81dbb619c5225dd0572,NaN,AK,BO,MX Amigos y Familia,Playa,180
25,3,18,10.500000,3,18,13.416667,AT-AI,85d3b182431d323b856e50e249689981,XA-VIY,AT,AI,Playa,Ciudad Principal,186
26,4,5,0.833333,4,5,4.750000,BM-AV,04e83b220f5513bd6cc636784dcddec6,XA-VXC,BM,AV,MX Amigos y Familia,Ciudad Fronteriza,240
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245744,3,21,6.333333,3,21,10.166667,AO-BT,a0ee9f7974186536fc197f59353a6442,XA-VAE,AO,BT,Playa,Ciudad Principal,186
245746,10,1,11.916667,10,1,15.166667,BA-BT,ab5bb628f7e10e2d8fbdf9e15de7bf4a,NaN,BA,BT,Playa,Ciudad Principal,186
245749,4,20,10.750000,4,20,14.000000,AW-BT,ef32da2731db80faa8b9f5030979a016,9H-MLV,AW,BT,Playa,Ciudad Principal,178
245751,10,4,10.000000,10,4,13.250000,AW-BT,3c15779202b13158f78e8a8afe377250,NaN,AW,BT,Playa,Ciudad Principal,240


In [43]:
# Realizamos las proyecciones a futuro de pasajeros utilizando el modelo MLP
passenger_forecast = model_mlp_p.predict(future_flights)

In [44]:
# Añadimos estas proyecciones al dataframe de vuelos futuros, y reiniciamos el índice
future_flights["Passengers"] = passenger_forecast
future_flights = future_flights.reset_index()
future_flights.drop(["index"], axis = 1, inplace = True)
future_flights

,Departure Month,Departure Day,Departure Time,Arrival Month,Arrival Day,Arrival Time,DS-AS,Flight_ID,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,Capacity,Passengers
0,2,16,17.166667,2,16,17.916667,AW-AS,6cfa1bbaa44f08fc7d3061f034a6a5ce,XA-VBV,AW,AS,MX Amigos y Familia,Ciudad Principal,220,182.393036
1,1,12,8.333333,1,12,9.333333,BA-AB,bb319ee8f62b5ae38e6f6da4dbff03cb,XA-VAV,BA,AB,Playa,Ciudad Principal,186,157.284199
2,5,18,14.166667,5,18,16.250000,AK-BO,d77570d39116f81dbb619c5225dd0572,NaN,AK,BO,MX Amigos y Familia,Playa,180,153.576064
3,3,18,10.500000,3,18,13.416667,AT-AI,85d3b182431d323b856e50e249689981,XA-VIY,AT,AI,Playa,Ciudad Principal,186,185.676516
4,4,5,0.833333,4,5,4.750000,BM-AV,04e83b220f5513bd6cc636784dcddec6,XA-VXC,BM,AV,MX Amigos y Familia,Ciudad Fronteriza,240,163.666455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123520,3,21,6.333333,3,21,10.166667,AO-BT,a0ee9f7974186536fc197f59353a6442,XA-VAE,AO,BT,Playa,Ciudad Principal,186,128.607949
123521,10,1,11.916667,10,1,15.166667,BA-BT,ab5bb628f7e10e2d8fbdf9e15de7bf4a,NaN,BA,BT,Playa,Ciudad Principal,186,115.971634
123522,4,20,10.750000,4,20,14.000000,AW-BT,ef32da2731db80faa8b9f5030979a016,9H-MLV,AW,BT,Playa,Ciudad Principal,178,124.512088
123523,10,4,10.000000,10,4,13.250000,AW-BT,3c15779202b13158f78e8a8afe377250,NaN,AW,BT,Playa,Ciudad Principal,240,155.995081


In [45]:
# Realizamos la proyección de ventas de productos utilizando el modelo Decision Tree
food_forecast = model_dt_f.predict(future_flights)

In [46]:
# Convertimos las labels a datos numéricos para tomarlos como cotas superiores
food_forecast_numeric = food_forecast.astype('float64')

In [47]:
# Convertimos los resultados a un dataframe
food_forecast_df = pd.DataFrame(food_forecast_numeric, columns = y_f.columns)
food_forecast_df

,Coca Cola Regular,Agua Natural 600 Ml,Sabritas Originales,Ruffles Queso,Doritos Nacho,Panini Clasico,Fritos Limon Y Sal,Sabritas Flamin Hot,Sprite,Cheetos,...,Maxi Combo,Heineken 0,Cuerno Individual Charter,Quaker Natural Balance,Quaker Avena Moras,Nissin Limon Y Habanero,Combo Stl,Club Sandwich,Kacang Flaming Hot,Gomita Enchilada La Cueva
0,8.0,8.0,0.0,8.0,8.0,4.0,4.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4.0,4.0,0.0,4.0,4.0,4.0,4.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,20.0,8.0,8.0,4.0,4.0,4.0,4.0,8.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,8.0,8.0,4.0,4.0,4.0,4.0,0.0,0.0,4.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123520,4.0,8.0,0.0,4.0,0.0,0.0,0.0,4.0,4.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
123521,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
123522,4.0,4.0,0.0,4.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
123523,20.0,8.0,12.0,8.0,8.0,8.0,8.0,4.0,4.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
# Combinamos ambos dataframes para tener la información completa
future_flights = pd.concat([future_flights, food_forecast_df], axis=1)
future_flights

,Departure Month,Departure Day,Departure Time,Arrival Month,Arrival Day,Arrival Time,DS-AS,Flight_ID,Aeronave,DepartureStation,...,Maxi Combo,Heineken 0,Cuerno Individual Charter,Quaker Natural Balance,Quaker Avena Moras,Nissin Limon Y Habanero,Combo Stl,Club Sandwich,Kacang Flaming Hot,Gomita Enchilada La Cueva
0,2,16,17.166667,2,16,17.916667,AW-AS,6cfa1bbaa44f08fc7d3061f034a6a5ce,XA-VBV,AW,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,12,8.333333,1,12,9.333333,BA-AB,bb319ee8f62b5ae38e6f6da4dbff03cb,XA-VAV,BA,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5,18,14.166667,5,18,16.250000,AK-BO,d77570d39116f81dbb619c5225dd0572,NaN,AK,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,18,10.500000,3,18,13.416667,AT-AI,85d3b182431d323b856e50e249689981,XA-VIY,AT,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,5,0.833333,4,5,4.750000,BM-AV,04e83b220f5513bd6cc636784dcddec6,XA-VXC,BM,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123520,3,21,6.333333,3,21,10.166667,AO-BT,a0ee9f7974186536fc197f59353a6442,XA-VAE,AO,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
123521,10,1,11.916667,10,1,15.166667,BA-BT,ab5bb628f7e10e2d8fbdf9e15de7bf4a,NaN,BA,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
123522,4,20,10.750000,4,20,14.000000,AW-BT,ef32da2731db80faa8b9f5030979a016,9H-MLV,AW,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
123523,10,4,10.000000,10,4,13.250000,AW-BT,3c15779202b13158f78e8a8afe377250,NaN,AW,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#Preprocesamiento


In [49]:
productos = pd.read_csv("/content/Sales TEC_Valid.csv")
productos

,Flight_ID,ProductType,ProductName,Quantity,TotalSales
0,a05290288259526edd3601160b10e1de,Botanas,Carne Seca Habanero,1,55.0
1,08f6f97437df8db101b050f1110be656,Licores,Jw Red Label,2,240.0
2,4e09c949826a77207868412baeff6d30,Licores,Jack And Coke,8,576.0
3,c3e1568fe46c68d3174681d322d412b3,Licores,Jw Red Label,2,240.0
4,7396e43fb94caca87add7bb1f1611e7e,Vivabus,Vivabus_Gdl.-Nvo. C. Camionera,4,320.0
...,...,...,...,...,...
2564356,638cf1904f936c50aea4a6f123ee07cf,Licores,Vino Tinto Sangre De Toro,1,155.0
2564357,04c5a19b8382a888d5c4c7b4932e783e,Licores,Vino Tinto Sangre De Toro,1,155.0
2564358,0291f6b201d2b24717a8ed7ca04f2d90,Licores,Vino Tinto Sangre De Toro,1,155.0
2564359,a10e3ac614fd434d6312897e07727c08,Licores,Vino Tinto Sangre De Toro,1,155.0


In [50]:
productos = productos[~productos['ProductName'].str.contains("Combo")]
productos_alimentos = productos[productos['ProductType'].isin(["Botanas","Licores","Galletas","Bebidas Calientes","Alimentos Charter","Perecederos","Refrescos","Sopas","Lacteos"])]

In [51]:
productos_alimentos

,Flight_ID,ProductType,ProductName,Quantity,TotalSales
0,a05290288259526edd3601160b10e1de,Botanas,Carne Seca Habanero,1,55.0
1,08f6f97437df8db101b050f1110be656,Licores,Jw Red Label,2,240.0
2,4e09c949826a77207868412baeff6d30,Licores,Jack And Coke,8,576.0
3,c3e1568fe46c68d3174681d322d412b3,Licores,Jw Red Label,2,240.0
5,34ae909bf3699372fcd66f02af0bdd54,Licores,Ron Bacardi,2,240.0
...,...,...,...,...,...
2564356,638cf1904f936c50aea4a6f123ee07cf,Licores,Vino Tinto Sangre De Toro,1,155.0
2564357,04c5a19b8382a888d5c4c7b4932e783e,Licores,Vino Tinto Sangre De Toro,1,155.0
2564358,0291f6b201d2b24717a8ed7ca04f2d90,Licores,Vino Tinto Sangre De Toro,1,155.0
2564359,a10e3ac614fd434d6312897e07727c08,Licores,Vino Tinto Sangre De Toro,1,155.0


##Botanas

In [52]:
valores_unicos_botanas = productos_alimentos[productos_alimentos['ProductType']=="Botanas"]["ProductName"].unique()
valores_unicos_botanas

array(['Carne Seca Habanero', 'Arandano Mango Mix', 'Go Nuts', 'Arandano',
       'Nutty Berry Mix', 'Frutos Secos Enchilados', 'Dip De Queso',
       'Nueces De Arbol Mix', 'Carne Seca Original', 'Luxury Nut Mix',
       'Tostitos Nachos Con Dip', 'Protein Adventure', 'Salsa Botanera',
       'Eco Holder', 'Cheetos', 'Tostitos', 'Rancheritos',
       'Doritos Nacho', 'Mafer Sin Sal', 'Ruffles Queso',
       'Nishikawa Salado', 'Gomita Enchilada La Cueva',
       'Kacang Flaming Hot', 'Nishikawa Japones', 'Cheetos Flamin Hot',
       'Fritos Limon Y Sal', 'Sabritas Flamin Hot', 'Sabritas Originales'],
      dtype=object)

In [53]:
cajon3 = ['Mafer Sin Sal','Carne Seca Habanero', 'Arandano Mango Mix', 'Go Nuts', 'Arandano',
       'Nutty Berry Mix', 'Frutos Secos Enchilados',
       'Nueces De Arbol Mix', 'Carne Seca Original', 'Luxury Nut Mix',
       'Nishikawa Salado', 'Nishikawa Japones','Kacang Flaming Hot','Gomita Enchilada La Cueva']

cajon4 = [
       'Tostitos Nachos Con Dip', 'Cheetos', 'Tostitos', 'Rancheritos',
       'Doritos Nacho', 'Ruffles Queso',  'Cheetos Flamin Hot',
       'Fritos Limon Y Sal', 'Sabritas Flamin Hot', 'Sabritas Originales']

In [54]:
len(cajon4)+len(cajon3)

24

In [55]:
cajon34=np.concatenate((cajon4,cajon3))
np.setdiff1d(np.union1d(cajon34, valores_unicos_botanas), np.intersect1d(cajon34, valores_unicos_botanas))

array(['Dip De Queso', 'Eco Holder', 'Protein Adventure',
       'Salsa Botanera'], dtype=object)

##Galletas

In [56]:
valores_unicos_galletas = productos_alimentos[productos_alimentos['ProductType']=="Galletas"]["ProductName"].unique()
valores_unicos_galletas

array(['Muffin Integral', 'Quaker Granola', 'Quaker Avena Frutos Rojos',
       'Galleta De Chispas De Chocolate', 'Galleta De Chocolate',
       'Galleta De Arandano Relleno De Q/Crema', 'Quaker Avena Moras',
       'Quaker Natural Balance', 'Chokis', 'Arcoiris',
       'Emperador Vainilla', 'Emperador Chocolate'], dtype=object)

In [57]:
cajon3 = np.concatenate((valores_unicos_galletas,cajon3))

##Licores

In [58]:
valores_unicos_licor = productos_alimentos[productos_alimentos['ProductType']=="Licores"]["ProductName"].unique()
valores_unicos_licor=np.delete(valores_unicos_licor, np.where(valores_unicos_licor == 'Jw Red Label ') )
valores_unicos_licor=np.delete(valores_unicos_licor, np.where(valores_unicos_licor == 'Baileys ') )


In [59]:
caja1_l = ['Vino Tinto Sangre De Toro','Vino Blanco Cria Cuervos ', 'Corajillo Baileys ',
       'Vino Tinto Cria Cuervos', 'Corajillo Baileys ','Ron Bacardi',
       'Baileys', 'Corajillo', 'Tequila 7 Leguas Reposado',
       'Tequila 7 Leguas Blanco','Licor Charter']

In [60]:
len(caja1_l)

11

In [61]:
carro4 = np.setdiff1d(np.union1d(caja1_l, valores_unicos_licor), np.intersect1d(caja1_l, valores_unicos_licor))

In [62]:
carro4

array(['Amstel Ultra', 'Cerveza Charter', 'Heineken 0',
       'Heineken Original', 'Heineken Silver', 'Jack And Coke',
       'Jw Red Label', 'Sol Clamato', 'Tecate Light',
       'Topochico Seltzer Fresa-Guayaba', 'Topochico Seltzer Mango',
       'Ultra Seltzer Frambuesa', 'Xx Lager', 'Xx Ultra'], dtype=object)

##Bebidas Calientes

In [63]:
valores_unicos_caliente = productos_alimentos[productos_alimentos['ProductType']=="Bebidas Calientes"]["ProductName"].unique()
valores_unicos_caliente

array(['Te Relax', 'Te Frutos Rojos', 'Te Manzanilla Jengibre',
       'Te Vainilla', 'Cafe De Olla', 'Cafe 19 Cafe Clasico',
       'Cafe Costa', 'Cafe 19 Chiapas', 'Cafe 19 Capuchino'], dtype=object)

In [64]:
cajon_te=['Te Relax', 'Te Frutos Rojos', 'Te Manzanilla Jengibre',
       'Te Vainilla']
cajon_cafe = ['Cafe De Olla', 'Cafe 19 Cafe Clasico',
       'Cafe Costa', 'Cafe 19 Chiapas', 'Cafe 19 Capuchino']

##Perecederos

In [65]:
valores_unicos_charter = productos_alimentos[productos_alimentos['ProductType']=="Alimentos Charter"]["ProductName"].unique()
valores_unicos_charter

array(['Cuerno Individual Charter', 'Club Sandwich'], dtype=object)

In [66]:
valores_unicos_pere = productos_alimentos[productos_alimentos['ProductType']=="Perecederos"]["ProductName"].unique()
valores_unicos_pere

array(['Panini Clasico', 'Panini Integral', 'Mega Cuerno Clasico',
       'Cuerno Clasico De Pavo'], dtype=object)

In [67]:
perecederos = np.concatenate((valores_unicos_charter,valores_unicos_pere))
perecederos

array(['Cuerno Individual Charter', 'Club Sandwich', 'Panini Clasico',
       'Panini Integral', 'Mega Cuerno Clasico', 'Cuerno Clasico De Pavo'],
      dtype=object)

##Refrescos

In [68]:
valores_unicos_bebidas = productos_alimentos[productos_alimentos['ProductType']=="Refrescos"]["ProductName"].unique()
valores_unicos_bebidas

array(['Sprite', 'Jugo De Mango', 'Sidral Mundet', 'Coca Cola Dieta',
       'Coca Sin Azucar', 'Jugo De Manzana', 'Fanta De Naranja',
       'Ciel Mineralizada', 'Coca Cola Regular', 'Agua Natural 600 Ml'],
      dtype=object)

In [69]:
refrescos =['Sprite',  'Sidral Mundet', 'Coca Cola Dieta',
       'Coca Sin Azucar',  'Fanta De Naranja',
       'Ciel Mineralizada', 'Coca Cola Regular']
jugo =['Jugo De Mango','Jugo De Manzana']
agua = ['Agua Natural 600 Ml']

##Sopas

In [70]:
valores_unicos_sopas = productos_alimentos[productos_alimentos['ProductType']=="Sopas"]["ProductName"].unique()
valores_unicos_sopas

array(['Nissin Res', 'Nissin Fuego', 'Nissin Picante',
       'Nissin Limon Y Habanero', 'Nissin Dark Dragon'], dtype=object)

##Lacteos

In [71]:
valores_unicos_lacteos = productos_alimentos[productos_alimentos['ProductType']=="Lacteos"]["ProductName"].unique()
valores_unicos_lacteos

array(['Leche De Fresa Sc', 'Leche De Chocolate Sc'], dtype=object)

#Modelacion

##Crear cajones

In [72]:
cajon3

array(['Muffin Integral', 'Quaker Granola', 'Quaker Avena Frutos Rojos',
       'Galleta De Chispas De Chocolate', 'Galleta De Chocolate',
       'Galleta De Arandano Relleno De Q/Crema', 'Quaker Avena Moras',
       'Quaker Natural Balance', 'Chokis', 'Arcoiris',
       'Emperador Vainilla', 'Emperador Chocolate', 'Mafer Sin Sal',
       'Carne Seca Habanero', 'Arandano Mango Mix', 'Go Nuts', 'Arandano',
       'Nutty Berry Mix', 'Frutos Secos Enchilados',
       'Nueces De Arbol Mix', 'Carne Seca Original', 'Luxury Nut Mix',
       'Nishikawa Salado', 'Nishikawa Japones', 'Kacang Flaming Hot',
       'Gomita Enchilada La Cueva'], dtype=object)

In [73]:
cajon_bebidas = np.concatenate((refrescos, jugo, valores_unicos_lacteos))
cajon_botana = np.array(cajon3)
cajon_papas = np.array(cajon4)
cajon_agua = np.array(agua)
cajon_te = np.array(cajon_te)
cajon_cafe = np.array(cajon_cafe)
cajon_licorLata = np.array(carro4)
cajon_licorBotella = np.array(caja1_l)
cajon_perecederos = np.array(perecederos)

##Capacidades por producto

In [74]:
capacidad_bebidas = 100
capacidad_botana = 90
capacidad_papas = 120
capacidad_agua = 50
capacidad_te = 12
capacidad_cafe = 35
capacidad_licorLata = 80
capcidad_licorBotella = 23
capacidad_perecederos = 24

cantidad total de productos

In [75]:
total_productos = np.concatenate((cajon_bebidas,cajon_botana,cajon_papas,cajon_agua,cajon_te,cajon_cafe,cajon_licorLata,cajon_licorBotella,cajon_perecederos))

In [76]:
len(total_productos)

88

In [77]:
total_productos

array(['Sprite', 'Sidral Mundet', 'Coca Cola Dieta', 'Coca Sin Azucar',
       'Fanta De Naranja', 'Ciel Mineralizada', 'Coca Cola Regular',
       'Jugo De Mango', 'Jugo De Manzana', 'Leche De Fresa Sc',
       'Leche De Chocolate Sc', 'Muffin Integral', 'Quaker Granola',
       'Quaker Avena Frutos Rojos', 'Galleta De Chispas De Chocolate',
       'Galleta De Chocolate', 'Galleta De Arandano Relleno De Q/Crema',
       'Quaker Avena Moras', 'Quaker Natural Balance', 'Chokis',
       'Arcoiris', 'Emperador Vainilla', 'Emperador Chocolate',
       'Mafer Sin Sal', 'Carne Seca Habanero', 'Arandano Mango Mix',
       'Go Nuts', 'Arandano', 'Nutty Berry Mix',
       'Frutos Secos Enchilados', 'Nueces De Arbol Mix',
       'Carne Seca Original', 'Luxury Nut Mix', 'Nishikawa Salado',
       'Nishikawa Japones', 'Kacang Flaming Hot',
       'Gomita Enchilada La Cueva', 'Tostitos Nachos Con Dip', 'Cheetos',
       'Tostitos', 'Rancheritos', 'Doritos Nacho', 'Ruffles Queso',
       'Cheetos Fl

In [78]:
# Realizamos una nueva dataframe con la información util, y con los productos ordenados igual que el modelo matemático. Además dropeamos NaNs
future_flights_demand = pd.concat([future_flights[["Departure Month", "Departure Day", "Departure Time", "Aeronave", "DepartureStation"]], future_flights[total_productos]], axis = 1)
future_flights_demand.dropna(inplace = True)
future_flights_demand

,Departure Month,Departure Day,Departure Time,Aeronave,DepartureStation,Sprite,Sidral Mundet,Coca Cola Dieta,Coca Sin Azucar,Fanta De Naranja,...,Corajillo,Tequila 7 Leguas Reposado,Tequila 7 Leguas Blanco,Licor Charter,Cuerno Individual Charter,Club Sandwich,Panini Clasico,Panini Integral,Mega Cuerno Clasico,Cuerno Clasico De Pavo
0,2,16,17.166667,XA-VBV,AW,4.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0
1,1,12,8.333333,XA-VAV,BA,0.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0
3,3,18,10.500000,XA-VIY,AT,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,5,0.833333,XA-VXC,BM,4.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,4.0,0.0
5,2,24,7.583333,XA-VII,BA,0.0,0.0,0.0,4.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123508,4,14,11.416667,XA-VXO,BA,4.0,4.0,0.0,4.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,4.0,0.0
123509,3,21,9.916667,XA-VYE,AW,4.0,4.0,4.0,4.0,4.0,...,0.0,4.0,4.0,0.0,0.0,0.0,4.0,4.0,4.0,0.0
123513,1,27,12.833333,XA-VXD,AT,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,0.0
123520,3,21,6.333333,XA-VAE,AO,4.0,4.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0


In [79]:
# Esta función toma un dataframe como el anterior y obtiene el itinerario de algún avión aleatorio algún dia aleatorio, y lo retorna en el formato necesario para el modelo matemático
def generate_path(flights_demand):
  # Establecemos los aeropuertos de recarga
  supply_airports = ('AO', 'AU', 'AW', 'BA', 'BM', 'AT', 'AK')

  # Convertimos a arreglo de numpy para mayor eficiencia
  flights_array = flights_demand.to_numpy()

  # Seleccionamos un vuelo aleatorio, y obtenemos su fecha y aeronave
  random_flight = flights_demand.sample().to_numpy()[0]
  month = random_flight[0]
  day = random_flight[1]
  time = random_flight[2]
  aeronave = random_flight[3]
  print(aeronave, month, day)

  # Iteramos sobre el arreglo de vuelos, y si encontramos un vuelo de la misma aeronave el mismo dia, lo añadimos al camino
  path = []
  for row in flights_array:
    if row[0] == month and row[1] == day and row[3] == aeronave:
      path.append(row)

  # Convertimos a arreglo de numpy, sorteamos con respecto a la hora de salida, y dropeamos toda la información innecesaria
  path = np.array(path)
  path = path[np.argsort(path[:, 2])]
  path = path[:, 4:]

  # El siguiente procedimiento traslada la demanda de los vuelos que no salen de aeropuertos de recarga al vuelo más cercano anterior que sí salió de una, para tomar en cuenta la cantidad necesaria de recargar en ese aeropuerto de abastecimiento hasta llegar a otro aeropuerto de abastecimiento
  temp = np.zeros(88)
  for i in range(path.shape[0]-1, -1, -1):
    row = path[i, :]
    if row[0] in supply_airports:
      row[-88:] = row[-88:] + temp
      temp = np.zeros(88)
    else:
      temp = temp + row[-88:]
      row[-88:] = 0
  print(path.T[0])
  return path.T[1:]

test_path = generate_path(future_flights_demand)

XA-VXD 1 3
['AW' 'BD' 'AW']


In [80]:
test_path.shape

(88, 3)

cantidad por categoria

In [81]:
total_productos_longitud = np.array([len(cajon_bebidas),len(cajon_botana),len(cajon_papas),len(cajon_agua),len(cajon_te),len(cajon_cafe),len(cajon_licorLata),len(cajon_licorBotella),len(cajon_perecederos)])

In [82]:
total_productos_longitud

array([11, 26, 10,  1,  4,  5, 14, 11,  6])

en abastecimiento posible consideramos el inicio

In [83]:
abastecimiento_posible = test_path.shape[1]
lista_abastecimiento = np.arange(abastecimiento_posible)

modelo

In [84]:
matriz_random = test_path

matriz_con_fila_cero = np.vstack([matriz_random, np.zeros((1, matriz_random.shape[1]))])

matriz_con_fila_columna_cero = np.hstack([matriz_con_fila_cero, np.zeros((matriz_con_fila_cero.shape[0], 1))])

In [86]:
import os

os.environ['NEOS_EMAIL'] = 'sruelas261203@gmail.com'

model = pyo.ConcreteModel()
# SETS (used to index the decision variable and the parameters)
model.A = pyo.RangeSet(len(total_productos))
model.L = pyo.RangeSet(abastecimiento_posible)

model.x = pyo.Var(model.A, model.L, domain=pyo.NonNegativeIntegers)

def W_init(model, i, j):
    return matriz_con_fila_columna_cero[i,j]

model.salidas = pyo.Param(model.A, model.L, initialize = W_init,mutable=True)

# iniciamos con minimo uno de cada elemento
def limit_all(model, i):
    return model.x[i, 1] >= 1
model.C1 = pyo.Constraint(model.A, rule=limit_all)

# cumplir demanda
def suficiente(model, i, j ):
  diferencia = 0
  for n in range(1,j+1):
    diferencia += model.x[i, n]-model.salidas[i,n]
  return diferencia >= 0
model.C2 = pyo.Constraint(model.A,model.L, rule=suficiente)

# cumplir capacidad
def capacidad1(model, j):
  suma = 0
  for m in range(1,j+1):
    for n in range(1,12):
      suma += model.x[n,m]
  return suma <= 100
model.C3 = pyo.Constraint(model.L, rule=capacidad1)

def capacidad2(model, j):
  suma = 0
  for m in range(1,j+1):
    for n in range(12,38):
      suma += model.x[n,m]
  return suma <= 90
model.C4 = pyo.Constraint(model.L, rule=capacidad2)

def capacidad3(model, j):
  suma = 0
  for m in range(1,j+1):
    for n in range(38,48):
      suma += model.x[n,m]
  return suma <= 120
model.C5 = pyo.Constraint(model.L, rule=capacidad3)

def capacidad4(model, j):
  for m in range(1,j+1):
    return model.x[48,m] <= 50
model.C6 = pyo.Constraint(model.L, rule=capacidad4)

def capacidad5(model, j):
  suma = 0
  for m in range(1,j+1):
    for n in range(49,53):
      suma += model.x[n,m]
  return suma <= 12
model.C7 = pyo.Constraint(model.L, rule=capacidad5)

def capacidad6(model, j):
  suma = 0
  for m in range(1,j+1):
    for n in range(53,58):
      suma += model.x[n,m]
  return suma <= 35
model.C8 = pyo.Constraint(model.L, rule=capacidad6)

def capacidad7(model, j):
  suma = 0
  for m in range(1,j+1):
    for n in range(58,72):
      suma += model.x[n,m]
  return suma <= 80
model.C9 = pyo.Constraint(model.L, rule=capacidad7)

def capacidad8(model, j):
  suma = 0
  for m in range(1,j+1):
    for n in range(72,83):
      suma += model.x[n,m]
  return suma <= 23
model.C10 = pyo.Constraint(model.L, rule=capacidad8)

def capacidad9(model, j):
  suma = 0
  for m in range(1,j+1):
    for n in range(83,89):
      suma += model.x[n,m]
  return suma <= 24
model.C11 = pyo.Constraint(model.L, rule=capacidad9)




def objective(model):
  penalizacion = 1
  for j in model.L:
    suma=1
    for i in model.A:
      suma +=model.x[i,j]
    penalizacion += suma/(suma+1)

  desperdicio = 0
  for n in range(83,89):
    for j in model.L:
      desperdicio += model.x[n,j]-model.salidas[n,j]

  return (penalizacion)*100 + desperdicio*10
model.obj=pyo.Objective(rule=objective, sense=pyo.minimize)

#opt = pyo.SolverFactory('glpk')
#opt = pyo.SolverFactory("gurobi", solver_io="python")
#opt.solve(model)

solver_manager = pyo.SolverManagerFactory('neos')

results = solver_manager.solve(model, solver = "minos")

In [87]:
matriz_respuesta = np.zeros((len(total_productos), abastecimiento_posible))

for i in range(1,len(total_productos)+1):
  for j in range(1,abastecimiento_posible+1):
    matriz_respuesta[i-1,j-1]=model.x[i,j].value

print(matriz_respuesta)
matriz_minima = matriz_respuesta

[[ 1.  0.  0.]
 [ 4.  0.  0.]
 [ 4.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [12.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 4.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 4.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 4.  0.  0.]
 [ 4.  0.  0.]
 [ 1.  0.  0.]
 [ 4.  0.  0.]
 [ 4.  0.  0.]
 [ 8.  0.  0.]
 [ 8.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 4.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 4.  0.  0.]
 [ 4.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 4.  0.  0.]
 [ 1.  0.  0.]
 [ 4.  0. 

#Predicción 2024

In [94]:
future_flights_2 = future_flights.set_index("Flight_ID")
predicciones = future_flights_2.loc[['4d95c80c854b637e6fdf951e85f1a05b',  '2385ceefd5123f0cdf95994f9978d216',  '35b62c4a09ba0a69e0c9b9a006d83f18', 'c073f59d3bea0a295cc8e466265c28c1', 'b5b6665fdc1048a63fd7e2f224a51b6c']]
predicciones

,Departure Month,Departure Day,Departure Time,Arrival Month,Arrival Day,Arrival Time,DS-AS,Aeronave,DepartureStation,ArrivalStation,...,Maxi Combo,Heineken 0,Cuerno Individual Charter,Quaker Natural Balance,Quaker Avena Moras,Nissin Limon Y Habanero,Combo Stl,Club Sandwich,Kacang Flaming Hot,Gomita Enchilada La Cueva
Flight_ID,,,,,,,,,,,,,,,,,,,,,
4d95c80c854b637e6fdf951e85f1a05b,1,3,16.000000,1,3,17.666667,AW-AT,XA-VBM,AW,AT,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2385ceefd5123f0cdf95994f9978d216,1,4,16.333333,1,4,19.666667,AT-AK,XA-VCC,AT,AK,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35b62c4a09ba0a69e0c9b9a006d83f18,1,4,7.416667,1,4,9.666667,BB-BM,XA-VBR,BB,BM,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
c073f59d3bea0a295cc8e466265c28c1,1,6,19.333333,1,6,20.416667,AW-AP,XA-VBN,AW,AP,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
c073f59d3bea0a295cc8e466265c28c1,1,6,17.083333,1,6,18.666667,AK-AW,XA-VBN,AK,AW,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
b5b6665fdc1048a63fd7e2f224a51b6c,1,5,11.416667,1,5,12.333333,BH-AJ,XA-VBT,BH,AJ,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
b5b6665fdc1048a63fd7e2f224a51b6c,1,5,13.083333,1,5,14.333333,AJ-BM,XA-VBT,AJ,BM,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [98]:
predicciones.to_csv("out.csv")